In [1]:
import pandas as pd # load the libraries
import numpy as np

pd.set_option('display.max_columns', 100) # set the display for better viewing
pd.set_option('display.max_rows', 100)

raw_data = pd.read_csv('./data/train.csv') # load the data
raw_data.shape

C:\Users\heziy\AppData\Local\Temp\ipykernel_10904\812763184.py:7: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('./data/train.csv')


(150634, 77)

In [2]:
# create duplicate column for data verification
raw_data['test_date'] = pd.to_datetime(raw_data['Tranc_YearMonth'])

# Verify year data
raw_data['check'] = raw_data['test_date'].dt.year == raw_data['Tranc_Year']
raw_data['check'].value_counts()

True    150634
Name: check, dtype: int64

In [3]:
# Verify month data point
raw_data['check'] = raw_data['test_date'].dt.month == raw_data['Tranc_Month']
raw_data['check'].value_counts()

True    150634
Name: check, dtype: int64

In [4]:
# remove columns "Tranc_YearMonth" and dummy columns as it has been verified that they display correct information
raw_data.drop(['Tranc_YearMonth','check','test_date'], axis=1, inplace=True)
raw_data.shape

(150634, 76)

In [5]:
# we will use "planning area" for location analysis, so we will remove columns that are irrevelant
# column residential also removed as all data here are considered residential, offering no value
raw_data.drop(['town','block','street_name','address','postal','Latitude', 'Longitude','residential'], axis=1, inplace=True)
raw_data.shape

(150634, 68)

In [6]:
# verify if mid_storey is the same as mid
raw_data['test'] = raw_data['mid_storey'] == raw_data['mid']
raw_data['test'].value_counts()

True    150634
Name: test, dtype: int64

In [7]:
# remove columns 'mid_storey', as well as MRT and bus stop related information, leaving the nearest distance to the bus/mrt
raw_data.drop(['test','mid_storey',
               'mrt_name','mrt_latitude','mrt_longitude',
               'bus_stop_name', 'bus_stop_latitude','bus_stop_longitude'], axis=1, inplace=True)
raw_data.shape

(150634, 61)

In [8]:
# use floor_area_sqm, remove floor_area_sqft
raw_data.drop(['floor_area_sqft'], axis=1, inplace=True)
raw_data.shape

(150634, 60)

In [9]:
# removing columns flat_model and full_flat_type as I intend to focus on flat_type for the analysis
raw_data.drop(['flat_model','full_flat_type'], axis=1, inplace=True)
raw_data.shape

(150634, 58)

In [10]:
raw_data['diff'] = raw_data['lease_commence_date'] - raw_data['year_completed']
raw_data['diff'].value_counts().sort_index()

-10       16
-9        18
-6        12
-3         9
-1       430
 0     19931
 1     76255
 2     43586
 3      3339
 4       414
 5       918
 6      1463
 7      1208
 8       572
 9       891
 10      312
 11      497
 12       63
 13      293
 14       19
 15       22
 16       21
 17      108
 18       26
 20       68
 21       19
 24      114
 25       10
Name: diff, dtype: int64

In [11]:
# it surprises me that some of the flats have a lease commencement date that is earlier than year completion
# from the data it looks like these flats tend to have both residential and commercial as "Y"
# It might be possible for the lease for commercial units to start (those ground floor shop houses) before the completion of the entire block
# However, in this project, I believe it is sufficient to focus on hdb_age as the determining factor for tenor of the HDB
# Thus removing columns lease_commence_date and year_completed

raw_data.drop(['lease_commence_date','year_completed', 'diff'], axis=1, inplace=True)
raw_data.shape

(150634, 56)

In [12]:
# remove schools coordinates information
raw_data.drop(['pri_sch_latitude','pri_sch_longitude',
               'sec_sch_latitude','sec_sch_longitude',
               ], axis=1, inplace=True)
raw_data.shape

(150634, 52)

In [13]:
# convert boolean value to 1 and 0
for col in raw_data.columns[12:16]:
    raw_data[col] = raw_data[col].map({'Y':1, 'N':0})


In [14]:
# fill in NaN value as 0, indicating no mall/hawker within the set distance
columns = ['Mall_Within_500m', 'Mall_Within_1km', 'Mall_Within_2km',
           'Hawker_Within_500m', 'Hawker_Within_1km', 'Hawker_Within_2km']
for col in columns:
    raw_data[col] = raw_data[col].fillna(0)

In [15]:
raw_data['storey_range'].unique()

array(['10 TO 12', '07 TO 09', '13 TO 15', '01 TO 05', '01 TO 03',
       '06 TO 10', '28 TO 30', '19 TO 21', '04 TO 06', '16 TO 18',
       '22 TO 24', '16 TO 20', '34 TO 36', '25 TO 27', '11 TO 15',
       '37 TO 39', '31 TO 33', '43 TO 45', '40 TO 42', '21 TO 25',
       '49 TO 51', '46 TO 48', '26 TO 30', '36 TO 40', '31 TO 35'],
      dtype=object)

In [16]:
# we can use mid of the storey range as an approximate to which level the unit is located
# therefore, we can keep the mid column, and drop lower, upper and storey_range
raw_data.drop(['storey_range','lower', 'upper'], axis=1, inplace=True)
raw_data.shape

(150634, 49)

In [17]:
# combine columns that indicate number of units sold to the percenrage of total units sold over total units available
raw_data['total_unit_sold_%'] = raw_data[raw_data.columns[14:22]].sum(axis=1)*100/raw_data['total_dwelling_units']

# combine columns that indicate number of units rented to the percenrage of total units rental over total units available
raw_data['total_unit_rental_%'] = raw_data[raw_data.columns[22:26]].sum(axis=1)*100/raw_data['total_dwelling_units']

raw_data.drop(raw_data.columns[14:26], axis=1, inplace=True)
raw_data.shape

(150634, 39)

In [18]:
# going to focus on cutoff_point to determine the "prestige" of the school
# therefore not going to focus on the name of any schools at this point
raw_data.drop(['pri_sch_name','sec_sch_name'], axis=1, inplace=True)
raw_data.shape

(150634, 37)

In [19]:
raw_data['Mall_Nearest_Distance'].isnull().sum()

829

In [20]:
# 829 / 150634 = 0.55%, unlikely to be significant, remove
cleaned_data = raw_data.dropna()

In [21]:
cleaned_data.head()

,id,flat_type,floor_area_sqm,resale_price,Tranc_Year,Tranc_Month,mid,hdb_age,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,cutoff_point,affiliation,total_unit_sold_%,total_unit_rental_%
0,88471,4 ROOM,90.0,680000.0,2016,5,11,15,25,0,0,0,0,142,Kallang,1094.090418,0.0,0.0,7.0,154.753357,1.0,3.0,13.0,84,60,330.083069,0,0,29.427395,1138.633422,78,1,1138.633422,224,0,100.0,0.0
1,122598,5 ROOM,130.0,665000.0,2012,7,8,34,9,0,0,0,0,112,Bishan,866.941448,0.0,1.0,3.0,640.151925,0.0,1.0,7.0,80,77,903.659703,1,1,58.207761,415.607357,45,1,447.894399,232,0,100.0,0.0
2,170897,EXECUTIVE,144.0,838000.0,2013,7,14,24,16,0,0,0,0,90,Bukit Batok,1459.579948,0.0,0.0,4.0,1762.082341,0.0,0.0,1.0,84,95,1334.251197,1,0,214.747860,498.849039,39,0,180.074558,188,0,100.0,0.0
3,86070,4 ROOM,103.0,550000.0,2012,4,3,29,11,1,0,0,0,75,Bishan,950.175199,0.0,1.0,4.0,726.215262,0.0,1.0,9.0,32,86,907.453484,1,1,43.396521,389.515528,20,1,389.515528,253,1,100.0,0.0
4,153632,4 ROOM,83.0,298000.0,2017,12,2,34,4,0,0,0,0,48,Yishun,729.771895,0.0,1.0,2.0,1540.151439,0.0,0.0,1.0,45,0,412.343032,0,0,129.422752,401.200584,74,0,312.025435,208,0,100.0,0.0


In [22]:
cleaned_data.to_csv('./data/cleaned_data.csv')